In [87]:
import numpy as np

class site:
    def __init__(self, position, properties):
        self.index = position
        for name,value in properties.items():
            setattr(self, name, value)
        

class Ising:
    def __init__(self, inL, inh0):
        self.L = inL
        self.h0 = inh0
        self.rstep = 0
        self.system = []
        self.Omegalist = []
        self.hlist = [[0]]
        self.jlist = [[0]]
        self.lslist = [[0]]
        self.lblist = [[0]]
        self.mlist = [[0]]
        for i in range(1,self.L):
            self.system.append(site(i, {'Jright':np.random.uniform(0,1), 'h':np.random.uniform(0,self.h0), 'ls':.5, 'lb':.5, 'm':.5}))
        self.system.append(site(self.L-1, {'Jright':0, 'h':np.random.uniform(0,self.h0), 'ls':.5, 'lb':.5, 'm':.5}))
        
        self.hlist[0][0] = self.system[0].h
        self.jlist[0][0] = self.system[0].Jright
        self.lslist[0][0] = self.system[0].ls
        self.lblist[0][0] = self.system[0].lb
        self.mlist[0][0] = self.system[0].m
        for i in range(1,self.L-1):
            self.hlist[0].append(self.system[i].h)
            self.jlist[0].append(self.system[i].Jright)
            self.lslist[0].append(self.system[i].ls)
            self.lblist[0].append(self.system[i].lb)
            self.mlist[0].append(self.system[i].m)
        self.hlist[0].append(self.system[self.L-1].h)
        self.lslist[0].append(self.system[self.L-1].ls)
        self.mlist[0].append(self.system[self.L-1].m)
    
    
    def renormStep(self):
        bond = None
        Omega = 0
        OmegaPos = None
        
        #finding max energy scale
        for i in range(self.L):
            if Omega<self.system[i].Jright:
                Omega = self.system[i].Jright
                OmegaPos = i
                bond = True
            if Omega<self.system[i].h:
                Omega = self.system[i].h
                OmegaPos = i
                bond = False
        self.Omegalist.append(Omega)
        
        #decimating
        if bond:
            self.system[OmegaPos].h = self.system[OmegaPos+1].h * self.system[OmegaPos].h / Omega #hright*hleft/Omega
            self.system[OmegaPos].ls = self.system[OmegaPos].ls + self.system[OmegaPos+1].ls + self.system[OmegaPos].lb
            self.system[OmegaPos].lb = self.system[OmegaPos+1].lb
            self.system[OmegaPos].m = self.system[OmegaPos].m + self.system[OmegaPos+1].m
            self.system[OmegaPos].Jright = self.system[OmegaPos+1].Jright
            del self.system[OmegaPos+1]
        elif OmegaPos<(self.L-1) and OmegaPos != 0:
            self.system[OmegaPos-1].Jright = self.system[OmegaPos-1].Jright*self.system[OmegaPos].Jright/Omega #jleft*jright/Omega
            self.system[OmegaPos-1].lb = self.system[OmegaPos-1].lb + self.system[OmegaPos].ls + self.system[OmegaPos].lb
            del self.system[OmegaPos]
        elif OmegaPos == 0:
            del self.system[OmegaPos]
        else:
            self.system[OmegaPos-1].Jright = 0
            del self.system[OmegaPos]
        
        #book keeping
        self.L -= 1
        self.rstep += 1
        
        #adding new distribution info
        self.hlist.append([0])
        self.hlist[self.rstep][0] = self.system[0].h
        self.jlist.append([0])
        self.jlist[self.rstep][0] = self.system[0].Jright
        self.lslist.append([0])
        self.lslist[self.rstep][0] = self.system[0].ls
        self.lblist.append([0])
        self.lblist[self.rstep][0] = self.system[0].lb
        self.mlist.append([0])
        self.mlist[self.rstep][0] = self.system[0].m
        for i in range(1,self.L-1):
            self.hlist[self.rstep].append(self.system[i].h)
            self.jlist[self.rstep].append(self.system[i].Jright)
            self.lslist[self.rstep].append(self.system[i].ls)
            self.lblist[self.rstep].append(self.system[i].lb)
            self.mlist[self.rstep].append(self.system[i].m)
        self.hlist[self.rstep].append(self.system[self.L-1].h)
        self.lslist[self.rstep].append(self.system[self.L-1].ls)
        self.mlist[self.rstep].append(self.system[self.L-1].m)
        
    #Analytic Output
    def printCouplings(self):
        for i in range(self.L-1):
              print('h' + str(i) + ' = ' + str(self.system[i].h) + '\nJ' + str(i) + ',' + str(i+1) + ' = ' + str(self.system[i].Jright))
        print('h' + str(self.L-1) + ' = ' + str(self.system[self.L-1].h))
        print()
    
    def printTracked(self, step = None):
        if step == None:
            step = self.rstep
        hlistlog = np.log(self.hlist[step])
        jlistlog = np.log(self.jlist[step])
        print('log average h is: ' + str(np.mean(hlistlog)))
        print('log h variance is: ' + str(np.var(hlistlog)) + '\n')
        print('log average J is: ' + str(np.mean(jlistlog)))
        print('log J variance is: ' + str(np.var(jlistlog)) + '\n')
        print('average spin cluster size is: ' + str(np.mean(self.lslist[step])) + '\n')
        print('average bond length is: ' + str(np.mean(self.lblist[step])) + '\n')
        print('average magnetic moment is: ' + str(np.mean(self.mlist[step]))+ '\n')
        print('energy scale is: ' + str(self.Omegalist[step])+ '\n')
    
    def energyLengthScaling(self, startStep = 0, stopStep = None):
        #seems to not actually be correct
        if stopStep == None:
            stopStep = self.rstep
        VI = np.var(np.log(self.hlist[0])) + np.var(np.log(self.jlist[0]))
        for i in range(startStep, stopStep+1):
            print('log averaged h scales at renorm level ' + str(i) + ' with coefficient: ' + str(-np.mean(np.log(self.hlist[i]))/np.sqrt(VI * np.mean(self.lslist[i]))) + '\n')
            print('log averaged J scales at renorm level ' + str(i) + ' with coefficient: ' + str(-np.mean(np.log(self.jlist[i]))/np.sqrt(VI * np.mean(self.lblist[i]))) + '\n')
    
    def deltaTest(self, verbose = False, startStep = 0, stopStep = None):
        if stopStep == None:
            stopStep = self.rstep
        delta = []
        for i in range(startStep, stopStep+1):
            if verbose:
                print('delta at level ' + str(i) + ' would be: ' + str((np.mean(np.log(self.hlist[i])) - np.mean(np.log(self.jlist[i])))/(np.var(np.log(self.hlist[i])) + np.var(np.log(self.jlist[i])))) + '\n')
            delta.append((np.mean(np.log(self.hlist[i])) - np.mean(np.log(self.jlist[i])))/(np.var(np.log(self.hlist[i])) + np.var(np.log(self.jlist[i]))))
        print('delta, over levels ' + str(startStep) + '-' + str(stopStep) + ', has variance: ' + str(np.var(delta)) + '\n')
    
    def momentPerLtest(self, startStep = 0, stopStep = None):
        #not working either, maybe I'm messing something up
        if stopStep == None:
            stopStep = self.rstep
        for i in range(startStep, stopStep+1):
            print('Magnetization scaling coefficient: ' + str(np.average(np.divide(self.mlist[i],self.lslist[i]))/(np.absolute((np.mean(np.log(self.hlist[0])) - np.mean(np.log(self.jlist[0])))/(np.var(np.log(self.hlist[0])) + np.var(np.log(self.jlist[0]))))**(3/2-np.sqrt(5)/2))) + '\n')
    
L0 = 2000
tester = Ising(L0,.95)

#tester.printCouplings()
#tester.printDistributions()
for i in range(L0-400):
    tester.renormStep()
#tester.printCouplings()
#tester.printTracked()
#tester.energyLengthScaling()
#tester.deltaTest(True)
tester.momentPerLtest()

Magnetization scaling coefficient: 3.2322551302483804

Magnetization scaling coefficient: 3.2317161515706667

Magnetization scaling coefficient: 3.2311766333747576

Magnetization scaling coefficient: 3.230636574850159

Magnetization scaling coefficient: 3.230095975184754

Magnetization scaling coefficient: 3.229554833564798

Magnetization scaling coefficient: 3.2290131491749117

Magnetization scaling coefficient: 3.2284709211980815

Magnetization scaling coefficient: 3.227928148815652

Magnetization scaling coefficient: 3.2273848312073232

Magnetization scaling coefficient: 3.226840967551147

Magnetization scaling coefficient: 3.2262965570235216

Magnetization scaling coefficient: 3.2257515987991887

Magnetization scaling coefficient: 3.225206092051227

Magnetization scaling coefficient: 3.224660035951053

Magnetization scaling coefficient: 3.2241134296684097

Magnetization scaling coefficient: 3.2235662723713685

Magnetization scaling coefficient: 3.2230185632263226

Magnetization sca

Magnetization scaling coefficient: 3.078663059443873

Magnetization scaling coefficient: 3.0778984566814174

Magnetization scaling coefficient: 3.0778025233852286

Magnetization scaling coefficient: 3.0770364344650716

Magnetization scaling coefficient: 3.0769398453575527

Magnetization scaling coefficient: 3.076843135964719

Magnetization scaling coefficient: 3.076746306061738

Magnetization scaling coefficient: 3.076515013979733

Magnetization scaling coefficient: 3.0757457323246995

Magnetization scaling coefficient: 3.0749754902681

Magnetization scaling coefficient: 3.074204286010305

Magnetization scaling coefficient: 3.074105504232656

Magnetization scaling coefficient: 3.0733327912853357

Magnetization scaling coefficient: 3.072559111237956

Magnetization scaling coefficient: 3.072459113730749

Magnetization scaling coefficient: 3.072358990913382

Magnetization scaling coefficient: 3.0715832451352725

Magnetization scaling coefficient: 3.0714824472148754

Magnetization scaling 

Magnetization scaling coefficient: 2.8948167342506257

Magnetization scaling coefficient: 2.893637686940003

Magnetization scaling coefficient: 2.8924566729115977

Magnetization scaling coefficient: 2.892173034800298

Magnetization scaling coefficient: 2.890988823879498

Magnetization scaling coefficient: 2.8906605864686643

Magnetization scaling coefficient: 2.890194412226715

Magnetization scaling coefficient: 2.8899079292132965

Magnetization scaling coefficient: 2.889620965926595

Magnetization scaling coefficient: 2.889333521157869

Magnetization scaling coefficient: 2.8890455936943167

Magnetization scaling coefficient: 2.888757182319061

Magnetization scaling coefficient: 2.8875621307423756

Magnetization scaling coefficient: 2.887271985119896

Magnetization scaling coefficient: 2.8878890324927866

Magnetization scaling coefficient: 2.8875986732198053

Magnetization scaling coefficient: 2.8873078238889796

Magnetization scaling coefficient: 2.8868344869111846

Magnetization scal

Magnetization scaling coefficient: 2.626161687834942

Magnetization scaling coefficient: 2.625116424326337

Magnetization scaling coefficient: 2.6243429992232516

Magnetization scaling coefficient: 2.624941859230554

Magnetization scaling coefficient: 2.6251491022494595

Magnetization scaling coefficient: 2.62299497896973

Magnetization scaling coefficient: 2.6221491841063926

Magnetization scaling coefficient: 2.6226980733122174

Magnetization scaling coefficient: 2.621638970960117

Magnetization scaling coefficient: 2.621843300517357

Magnetization scaling coefficient: 2.6207803719393894

Magnetization scaling coefficient: 2.619873381082576

Magnetization scaling coefficient: 2.6190832110836526

Magnetization scaling coefficient: 2.6182247129490883

Magnetization scaling coefficient: 2.618824179908273

Magnetization scaling coefficient: 2.618029580231667

Magnetization scaling coefficient: 2.6167429114843235

Magnetization scaling coefficient: 2.6145443003213193

Magnetization scalin